In [12]:
!pip install llama-index-llms-gemini llama-index
!pip install llama-index-embeddings-gemini

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [13]:
# Import necessary packages and define helper function to render output of models to markdown
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [14]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Configure genai to use your API Key
import os

 # add your GOOGLE API key here
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [15]:
from llama_index.llms.gemini import Gemini
from llama_index.core.settings import Settings
# imports
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


llm = Gemini(
    model="models/gemini-1.5-flash",
    system_prompt="""**Pcychologist Questions Assistant Guidelines**
    you can ask more questions from each patient to measure their anxiety level
    please create some creative questions and analyze patient problem





"""
)

In [16]:
#i have created these data with llm
import csv
import random

# Define possible patient statements and psychologist responses
patient_statements = [
    "I've been feeling really down lately.",
    "I can't sleep at night.",
    "I've lost interest in things I used to enjoy.",
    "I feel overwhelmed all the time.",
    "I'm constantly worried about everything.",
    "I have trouble concentrating.",
    "I've been having nightmares.",
    "I feel like a failure.",
    "I don't feel like myself anymore.",
    "I've been isolating myself from others.",
    "I'm angry all the time.",
    "I'm thinking about hurting myself.", #Important: Handle this responsibly in a real application.
    "I feel hopeless.",
    "I'm afraid of failing.",
    "I don't know what to do.",
    "I've been having panic attacks.",
    "My relationships are strained.",
    "I'm struggling with my identity.",
    "I'm having flashbacks.",
    "I'm hearing voices." #Important: Handle this responsibly in a real application.

]

psychologist_responses = [
    "I hear you saying you're feeling down. Can you tell me more about that?",
    "That sounds incredibly difficult.  Let's explore that further.",
    "It takes courage to talk about these things.  Thank you for sharing.",
    "Tell me more about what's been happening.",
    "What are some of the things that contribute to feeling overwhelmed?",
    "It sounds like you're experiencing a lot of stress. Let's work on coping mechanisms.",
    "We can work together to develop strategies for managing your sleep.",
    "Have you considered any ways to help yourself cope with these feelings?",
    "I want to assure you that you are not alone in feeling this way.",
    "Let's explore the roots of these feelings.",
    "That's a very serious concern. We need to address that immediately. Is there someone you trust that you could confide in right now?",
    "What steps can we take to work towards a solution?",
    "What are your thoughts about exploring different coping techniques?",
    "Let's focus on identifying your strengths and resources.",
    "It's understandable that you feel this way given the circumstances.",
    "Remember, you're not alone in this. We can work through this together.",
    "Is there anything specific that triggers your anxiety?",
    "We'll explore options to improve your relationships."
]
directory = '/content/my_data'
os.makedirs(directory, exist_ok=True)
filename='patient_psychologist_conversation.csv'
file_path = os.path.join(directory, filename)

# Generate the dataset
data = []
for i in range(120):  # More than 100 samples
    patient_statement = random.choice(patient_statements)
    psychologist_response = random.choice(psychologist_responses)
    data.append([patient_statement, psychologist_response])


# Write the data to a CSV file
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Patient', 'Psychologist'])  # Header row
    writer.writerows(data)

print("CSV file 'patient_psychologist_conversation.csv' created successfully.")

CSV file 'patient_psychologist_conversation.csv' created successfully.


In [17]:
# Configure settings
Settings.embed_model = GeminiEmbedding(
    model_name="models/text-embedding-004", api_key=GOOGLE_API_KEY, title="this is a document"
)
Settings.llm = llm
Settings.chunk_size = 1024
Settings.chunk_overlap = 0

# Index configuration
index_name = "hack2"

In [18]:



# Necessary to use the latest OpenAI models that support function calling API

data = SimpleDirectoryReader(input_dir="/content/my_data").load_data()
index = VectorStoreIndex.from_documents(data)

In [19]:
from llama_index.core.retrievers import VectorIndexRetriever, BaseRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.llms import ChatMessage, MessageRole

def support(history, query):
    # Create a chat engine
    chat_engine = index.as_chat_engine(
        chat_mode="condense_question", llm=llm, verbose=True
    )

    response = chat_engine.chat(query)

    output = ""
    for token in response.response:
        if token is None:
            continue
        output += token

    return  output
    # if you are using gradio
    #yeild output




In [20]:
history=" "


for i in [0,1,2,3,4]:

  user_input = input("Enter your explanation of your situation: ")
  history +=support(history,user_input)
  print(history)




Enter your explanation of your situation: i have anxiety 
Querying with: i have anxiety 
 The provided text does not contain the phrase "I have anxiety".  Therefore, I cannot answer your question.

Enter your explanation of your situation: can you help me i cannot focus
Querying with: can you help me i cannot focus
 The provided text does not contain the phrase "I have anxiety".  Therefore, I cannot answer your question.
Several approaches can be used to address this.  We can explore what might be triggering this, work on coping mechanisms, or identify your strengths and resources.  There are also options to help you feel less alone in this experience.



KeyboardInterrupt: Interrupted by user

In [23]:
prompt=f"""
you should compute a patient anxiety level by its chat history with a psychological chatbot
and categorize its level into one of the following categories:
poor mild severe
Poor might suggest the user is experiencing a high level of anxiety.
Mild might suggest the user is experiencing a low to moderate level of anxiety.
Severe might suggest the user is experiencing a very high and potentially dangerous level of anxiety.
the patient's chat history is :
{history}
the patient's anxiety level is :
"""
model = genai.GenerativeModel('gemini-1.5-pro-latest')
response = model.generate_content(prompt)

In [24]:
to_markdown(response.text)

> **Mild**
> 
> The chatbot's response is supportive and offers helpful strategies.  It acknowledges the user's potential anxiety *without explicitly stating the user confirmed having anxiety.*  It focuses on coping mechanisms, exploring triggers, and finding support, which suggests a measured and helpful approach to a situation that *could* involve anxiety, but doesn't definitively confirm a severe level.  It's also a response that could be given to someone experiencing mild distress or difficulty, not necessarily severe anxiety.
> 
> Since we don't have any user input to go on, and the chatbot is responding in a general, but supportive way, classifying it as "mild" seems the most appropriate.


In [20]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [34]:
import gradio as gr
def main():
    demo = gr.ChatInterface(
        support,
        chatbot=gr.Chatbot(height=500, rtl=True),
        textbox=gr.Textbox(placeholder="explain your situation.", container=False, scale=7, text_align="rtl", rtl=True),
        title="",
        theme="soft",



    )
    demo.launch(share=False, inline=True, server_name='0.0.0.0', server_port=2950)


if __name__ == '__main__':
    main()


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>